In [ ]:
#text------>NER mentions----->candidate_generation------->Entity_Linking

In [ ]:
# - create a knowledge Base
# - train an entity linker

In [ ]:
import pandas as pd

In [2]:
df_gold = pd.read_json("./news_articles-gold.jsonl",lines=True)

In [3]:
companies_df = pd.read_json("./company_collection.json")

In [18]:
def load_entities():
    companies_df = pd.read_json("./company_collection.json")
    names = {}
    descriptions = {}
    for _,row in companies_df.iterrows():
        url = row["url"]
        name = row["name"]
        description = row["industry_label"]+" "+row["headquarters"]+" "+row["description"]
        names[url] = name
        descriptions[url] = description
    return names,descriptions

In [5]:
names,descriptions = load_entities()

In [6]:
#Setup Knowledge base of companies
import spacy
from spacy.kb import KnowledgeBase,InMemoryLookupKB
nlp = spacy.load('en_core_web_lg')

/Users/vibha/anaconda3/envs/delpai-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
kb= InMemoryLookupKB(vocab=nlp.vocab,entity_vector_length=300)

In [8]:
#Add the descriptions into the Knowledge base
for url,desc in descriptions.items():
    #print(url)
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=url,entity_vector=desc_enc,freq=342)
    

In [ ]:
#Add the alias into the kb
for url,name in names.items():
    kb.add_alias(alias=name,entities=[url],probabilities=[1])

In [10]:
from pathlib import Path

In [70]:
# output_dir = Path.cwd().parent / "output"
# kb.to_disk(output_dir/"my_kb")
# nlp.to_disk(output_dir/"my_nlp")


In [ ]:
#how do you train this model

In [10]:
#Prepare dataset to train the model
dataset = []
for _,row in df_gold.iterrows():
    annotations = row["annotations"]
    text = row["text"]
    for entity,url in annotations.items():
        #print(url)
        start = text.find(entity)
        end = start+len(entity)
        offset = (start,end)
        links_dict = {url:1.0}
        dataset.append((text,{"links":{offset:links_dict}}))

In [22]:
dataset[:5]

[('Report: Adam Neumann, Benchmark Unloaded $676.5M In WeWork Shares Before Failed IPO. The more we learn about the WeWork debacle, the more head-scratching it gets. Subscribe to the Crunchbase Daily According to a news report from The Telegraph, WeWork co-founder and ousted CEO Adam Neumann unloaded $361 million worth of shares when Japanese investment conglomerate SoftBank first invested in the company in 2017. And he wasn’t the only one. The Telegraph reports that Benchmark , which was WeWorks’ first major investor, also cashed out at the time. It apparently “sold $315.5m in shares during the 2017 deal and a later SoftBank investment in 2019,” the news article reveals. Specifically, according to Crunchbase, Benchmark led WeWork’s $17 million Series A (ironically some may say) on April 1, 2012. This means that WeWork’s largest shareholders’ (previously undisclosed) sell-offs made up nearly one-third of the $2.3 billion worth of shares SoftBank bought from insiders right before the co

In [11]:
TRAIN_DOCS = []
for text, annotation in dataset:
    doc = nlp(text)
    TRAIN_DOCS.append((doc, annotation))

In [13]:
def create_kb(vocab):
        # print(vocab)
        # print(kb)
        # kb = InMemoryLookupKB(vocab=vocab, entity_vector_length=300)

        # for url,desc in descriptions.items():
        #     desc_doc = nlp(desc)
        #     desc_enc = desc_doc.vector
        #     kb.add_entity(entity=url,entity_vector=desc_enc,freq=342)
        # for url,name in names.items():
        #     kb.add_alias(alias=name,entities=[url],probabilities=[1])
        
        return kb
        
        

entity_linker = nlp.add_pipe("entity_linker", config={"incl_prior": False})
entity_linker.set_kb(create_kb)

In [ ]:
#Train the entity linking module
from spacy.util import minibatch, compounding
from spacy.training.example import Example
import random

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "entity_linker"]
with nlp.disable_pipes(*other_pipes):   # train only the entity_linker
    optimizer = nlp.create_optimizer()
    for itn in range(500):   # 500 iterations takes about a minute to train
        random.shuffle(TRAIN_DOCS)
        batches = minibatch(TRAIN_DOCS, size=compounding(4.0, 32.0, 1.001))  # increasing batch sizes
        losses = {}
        for batch in batches:
            for text,annotations in batch:
                #doc = nlp.make_doc(text)
                example = Example.from_dict(text, annotations)
                nlp.update([example], losses=losses, drop=0.3,sgd=optimizer)
        if itn % 50 == 0:
            print(itn, "Losses", losses)   # print the training loss
print(itn, "Losses", losses)

In [ ]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

In [17]:
# text = df_gold["text"][0]
# print(text)
# doc = nlp(text)
# for ent in doc.ents:
#     if ent.label_=="ORG":
#         print(ent.text, ent.label_, ent.kb_id)